# Homework 1. Link Prediction : CS6365 Fall 2025

Justin Mittereder - G49843234

## Part 1: Importing and Exploring the Dataset

In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd

In [27]:
#import nodes and edges
node_df = pd.read_csv('data/nodes.csv')
edge_df = pd.read_csv('data/train_edges.csv')
val_links = pd.read_csv('data/val_links.csv')
test_links = pd.read_csv('data/test_links.csv')

#print(node_df.head())
#print(node_df['label'].unique())
#print(edge_df.head())
#print(edge_df['relationship_type'].unique())

#print(type(node_df['properties'].iloc[0]))

#drop properties column because won't be used
node_df = node_df.drop('properties', axis=1)
#remove quotes and brackets so labels are simpler
node_df['label'] = node_df['label'].str.replace("['", "")
node_df['label'] = node_df['label'].str.replace("']", "")
print(node_df.columns)
print(node_df.head())
print()
print("Node Count: ", len(node_df))
print("Node Type Counts: ", node_df['label'].value_counts())

print()
print("Edge Type Counts: ", edge_df['relationship_type'].value_counts())

print()
print("Number of training examples: ", len(edge_df))
print("Number of validation examples: ", len(val_links))
print("Number of test examples: ", len(test_links))


Index(['id', 'label'], dtype='object')
   id    label
0   0  Dataset
1   1  Dataset
2   2  Dataset
3   3  Dataset
4   4  Dataset

Node Count:  5763
Node Type Counts:  label
Publication       2584
ScienceKeyword    1609
Dataset           1300
Platform           142
Instrument          83
Project             44
DataCenter           1
Name: count, dtype: int64

Edge Type Counts:  relationship_type
HAS_SCIENCEKEYWORD    4015
USES_DATASET          3623
SUBCATEGORY_OF        1823
HAS_PLATFORM          1519
OF_PROJECT            1325
HAS_DATASET           1300
HAS_INSTRUMENT         215
Name: count, dtype: int64

Number of training examples:  13820
Number of validation examples:  860
Number of test examples:  861


**Important: - `val_links.csv`: Contains `HAS_SCIENCEKEYWORD` edges for validation.`test_links.csv`: Contains `HAS_SCIENCEKEYWORD` edges for testing.** <br>
So, the validation and test sets are only looking at relationship type Dataset -> ScienceKeyword. 

In [6]:
# Map node IDs to contiguous indices for PyG
node_df['idx'] = range(len(node_df))
id_to_idx = dict(zip(node_df['id'], node_df['idx']))

ones = torch.ones((len(node_df), 1))  # dummy node features for each node
# Map node IDs to indices
edge_df['source'] = edge_df['source'].map(id_to_idx)
edge_df['target'] = edge_df['target'].map(id_to_idx)

val_links['source'] = val_links['source'].map(id_to_idx)
val_links['target'] = val_links['target'].map(id_to_idx)

test_links['source'] = test_links['source'].map(id_to_idx)
test_links['target'] = test_links['target'].map(id_to_idx)

# Create edge_index tensor
edge_index = torch.tensor(edge_df[['source', 'target']].values, dtype=torch.long).t().contiguous()

val_index = torch.tensor(val_links[['source', 'target']].values, dtype=torch.long).t().contiguous()
test_index = torch.tensor(test_links[['source', 'target']].values, dtype=torch.long).t().contiguous()

all_pos_edges = torch.cat([edge_index, val_index, test_index], dim=1) #for neg sampling later

#shape should be (2 , num_edges)
print(val_index.size())
print(test_index.size())

data = Data(x=ones, edge_index=edge_index)
print(data)

torch.Size([2, 860])
torch.Size([2, 861])
Data(x=[5763, 1], edge_index=[2, 13820])


In [7]:
#Summary of Dataset Structure and Key Statistics for Part 1
print("Number of Nodes: ", data.num_nodes)
print("Number of Node features: ", data.num_node_features)
print()
print("Number of Edges: ", data.num_edges)
print()
print("Has Isolated Nodes: " , data.has_isolated_nodes())
print("Has Self Loops: ", data.has_self_loops())

Number of Nodes:  5763
Number of Node features:  1

Number of Edges:  13820

Has Isolated Nodes:  False
Has Self Loops:  False


## Part 2: Link Prediction
### Method #1: Embedding-Based Approach

*Task*: Apply an embedding-based method for link prediction. *Description*: Train a model that generates node embeddings, then use those embeddings to predict links. Print relevant metrics.

"The primary goal of the NASA Knowledge Graph is to bridge scientific publications with the datasets they reference, facilitating deeper insights and research opportunities within NASA's scientific and data ecosystem. By organizing these interconnections within a graph structure, this dataset enables advanced analyses, such as discovering influential datasets, understanding research trends, and exploring scientific collaborations."

https://pytorch-geometric.readthedocs.io/en/2.6.0/tutorial/shallow_node_embeddings.

In [8]:
from torch_geometric.nn import Node2Vec

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=10, context_size=5, walks_per_node=10).to(device)

print(model)

loader = model.loader(batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Node2Vec(5763, 128)


In [9]:
for epoch in range(50):
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw, neg_rw)  # negative samples are neg_rw
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 1012.1948
Epoch 2, Loss: 737.2280
Epoch 3, Loss: 537.9211
Epoch 4, Loss: 400.6344
Epoch 5, Loss: 309.2363
Epoch 6, Loss: 247.5649
Epoch 7, Loss: 206.3209
Epoch 8, Loss: 180.3357
Epoch 9, Loss: 163.6973
Epoch 10, Loss: 153.8190
Epoch 11, Loss: 147.8310
Epoch 12, Loss: 143.8490
Epoch 13, Loss: 141.5399
Epoch 14, Loss: 139.8101
Epoch 15, Loss: 138.8957
Epoch 16, Loss: 138.1704
Epoch 17, Loss: 137.7518
Epoch 18, Loss: 137.5592
Epoch 19, Loss: 137.3433
Epoch 20, Loss: 137.3409
Epoch 21, Loss: 137.2219
Epoch 22, Loss: 137.2628
Epoch 23, Loss: 137.2513
Epoch 24, Loss: 137.4517
Epoch 25, Loss: 137.6578
Epoch 26, Loss: 137.6989
Epoch 27, Loss: 137.6433
Epoch 28, Loss: 137.5943
Epoch 29, Loss: 137.8565
Epoch 30, Loss: 137.6991
Epoch 31, Loss: 137.7848
Epoch 32, Loss: 137.8357
Epoch 33, Loss: 137.7349
Epoch 34, Loss: 137.7529
Epoch 35, Loss: 137.7550
Epoch 36, Loss: 137.7638
Epoch 37, Loss: 137.8042
Epoch 38, Loss: 137.6565
Epoch 39, Loss: 137.6357
Epoch 40, Loss: 137.5778
Epoch 41

In [11]:
#Generate negative samples for validation set and test set
from torch_geometric.utils import negative_sampling

print("val_index.size(1): ", val_index.size(1))
print("test_index.size(1): ", test_index.size(1))
print("model.size(0): ", model.num_nodes)

neg_val_index = negative_sampling(
    edge_index=all_pos_edges, 
    num_nodes=model.num_nodes,
    num_neg_samples=val_index.size(1)
)

neg_test_index = negative_sampling(
    edge_index=all_pos_edges,
    num_nodes=model.num_nodes,
    num_neg_samples=test_index.size(1)
)

print(neg_val_index)
print("neg_val_edge_index.shape: ", neg_val_index.shape)
print(neg_test_index)
print("neg_test_edge_index.shape: ", neg_val_index.shape)


val_index.size(1):  860
test_index.size(1):  861
model.size(0):  5763
tensor([[  60,  334, 3576,  ..., 2152, 2855, 4210],
        [2512,  974, 4610,  ...,  532,  680, 5609]])
neg_val_edge_index.shape:  torch.Size([2, 860])
tensor([[5535, 1363, 4188,  ..., 2220, 4556, 4670],
        [4425, 3079, 3996,  ..., 3792, 5148, 3171]])
neg_test_edge_index.shape:  torch.Size([2, 860])


In [39]:
embeddings = model()
#gets actual link predictions 
def predict_links(node_embeddings, edge_index):
    src, dst = edge_index
    score = (node_embeddings[src] * node_embeddings[dst]).sum(dim=1)  # dot product of two node embeddings
    #print(type(score))
    #print("Score: ", score)
    prob = torch.sigmoid(score) #make prob between 0 and 1
    threshold = 0.6
    pred = (prob > threshold).int()
    return pred

#gets scores from dot product of two node embeddings
def get_scores(node_embeddings, edge_index):
    src, dst = edge_index
    scores = (node_embeddings[src] * node_embeddings[dst]).sum(dim=1)  # dot product of two node embeddings
    return scores


#assemble true label tensors
val_true_labels = torch.cat([torch.ones(val_index.size(1)), torch.zeros(neg_val_index.size(1))])
test_true_labels = torch.cat([torch.ones(test_index.size(1)), torch.zeros(neg_test_index.size(1))])
print("val_true_labels.shape: " , val_true_labels.size())
print("test_true_labels.shape: ", test_true_labels.size())

#get 'presence of link' predictions for validation and test sets
pos_val_pred = predict_links(embeddings, val_index)
neg_val_pred = predict_links(embeddings, neg_val_index)
#print(val_true_labels)
print("pos_val_pred.shape: ", pos_val_pred.shape)
print("neg_val_pred.shape: ", neg_val_pred.shape)
pos_test_pred = predict_links(embeddings, test_index)
neg_test_pred = predict_links(embeddings, neg_test_index)

#assemble predicted label tensors
val_pred_labels = torch.cat([pos_val_pred, neg_val_pred])
test_pred_labels = torch.cat([pos_test_pred, neg_test_pred])

#get score predictions for validation and test sets
pos_val_scores = get_scores(embeddings, val_index)
neg_val_scores = get_scores(embeddings, neg_val_index)

pos_test_scores = get_scores(embeddings, test_index)
neg_test_scores = get_scores(embeddings, neg_test_index)

#assemble validation and test score tensors
val_scores = torch.cat([pos_val_scores, neg_val_scores])
test_scores = torch.cat([pos_test_scores, neg_test_scores])



val_true_labels.shape:  torch.Size([1720])
test_true_labels.shape:  torch.Size([1722])
pos_val_pred.shape:  torch.Size([860])
neg_val_pred.shape:  torch.Size([860])


In [41]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, recall_score, f1_score
validation_accuracy = accuracy_score(val_true_labels, val_pred_labels)
validation_recall = recall_score(val_true_labels, val_pred_labels)
validation_f1 = f1_score(val_true_labels, val_pred_labels)
print(f"Validation Accuracy: {validation_accuracy}, Validation Recall: {validation_recall}, Validation f1-score: {validation_f1}")
val_roc_auc = roc_auc_score(val_true_labels.cpu(), val_scores.detach().numpy())
val_avg_prec = average_precision_score(val_true_labels.cpu(), val_scores.detach().numpy())
print(f"Validation roc auc: {val_roc_auc}, Val Avg Precision: {val_avg_prec}")

print()

test_accuracy = accuracy_score(test_true_labels, test_pred_labels)
test_recall = recall_score(test_true_labels, test_pred_labels)
test_f1 = f1_score(test_true_labels, test_pred_labels)
print(f"Test Accuracy: {test_accuracy}, Test Recall: {test_recall}, Test f1-score: {test_f1}")
test_roc_auc = roc_auc_score(test_true_labels.cpu(), test_scores.detach().numpy())
test_avg_prec = average_precision_score(test_true_labels.cpu(), test_scores.detach().numpy())
print(f"Test roc auc: {test_roc_auc}, Test Avg Precision: {test_avg_prec}")

Validation Accuracy: 0.6970930232558139, Validation Recall: 0.5906976744186047, Validation f1-score: 0.6610279765777488
Validation roc auc: 0.6781408869659276, Val Avg Precision: 0.7723695715689489

Test Accuracy: 0.6945412311265969, Test Recall: 0.6039488966318235, Test f1-score: 0.6641123882503193
Test roc auc: 0.6762778877166571, Test Avg Precision: 0.7699010561281736


### Method 2: Alternative Approach 
*Task*: Choose and implement another link prediction method. *Description*: This method should not use embeddings. You can use any approach of your choice. Compare the performance of this method with the embedding-based method.

https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph <br>
https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html <br>
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.jaccard_coefficient.html

In [23]:
#Non-Embedding Method for link prediction
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt


#print(edge_df.head())
G = nx.from_pandas_edgelist(edge_df, source='source', target='target', edge_attr="relationship_type", create_using=nx.Graph())

all_val_edges = torch.cat([val_index, neg_val_index], dim=1) 
all_test_edges = torch.cat([test_index, neg_test_index], dim=1) 
#print(all_val_edges.shape)
#print(all_test_edges.shape)


def compute_jaccard(G, edges, threshold):
    scores = []
    edges = edges.t().tolist()
    for u, v in edges:
        try:
            preds = list(nx.jaccard_coefficient(G, [(u, v)]))
            scores.append(preds[0][2])
        except nx.NetworkXError:
            scores.append(0.0)

    pred_labels = []
    for score in scores: 
        if(score>threshold):
            pred_labels.append(1)
        else:
            pred_labels.append(0)
    return np.array(scores), np.array(pred_labels)


val_scores, val_pred_labels_jaccard = compute_jaccard(G, all_val_edges, 0.001)
test_scores, test_pred_labels_jaccard = compute_jaccard(G, all_test_edges, 0.001)

val_acc = accuracy_score(val_true_labels, val_pred_labels_jaccard)
val_recall = recall_score(val_true_labels, val_pred_labels_jaccard)
val_f1 = f1_score(val_true_labels, val_pred_labels_jaccard)
print()
print(f"Validation Accuracy: {val_acc}, Validation Recall: {val_recall}, Validation f1-score: {val_f1}")
val_auc = roc_auc_score(val_true_labels, val_scores)
val_precision = average_precision_score(val_true_labels, val_scores)
print(f"Validation roc auc: {val_auc}, Val Avg Precision: {val_precision}")

print()

test_acc = accuracy_score(test_true_labels, test_pred_labels_jaccard)
test_recall = recall_score(test_true_labels, test_pred_labels_jaccard)
test_f1 = f1_score(test_true_labels, test_pred_labels_jaccard)
print(f"Test Accuracy: {test_acc}, Test Recall: {test_recall}, Test f1-score: {test_f1}")
test_auc = roc_auc_score(test_true_labels, test_scores)
test_precision = average_precision_score(test_true_labels, test_scores)
print(f"Test roc auc: {test_auc}, Test Avg Precision: {test_precision}")


Validation Accuracy: 0.8261627906976744, Validation Recall: 0.7116279069767442, Validation f1-score: 0.8036769533814839
Validation roc auc: 0.8067212006489994, Val Avg Precision: 0.7102706604614362

Test Accuracy: 0.8147502903600464, Test Recall: 0.7015098722415796, Test f1-score: 0.7910936476751801
Test roc auc: 0.790084862023334, Test Avg Precision: 0.6830560086884887


## Part 3: Reflection and Analysis
*Task*: Compare both methods. *Description*: Write a reflection on the performance of each method. Discuss any challenges and insights gained. Suggest improvements to the dataset or the methods used.

### Intro and Approach
<p>The methods I used to predict the presence of links between Dataset and ScienceKeyword nodes in the nasa-gesdisc dataset were the Jaccard coefficient and Node2Vec. I predicted that Node2Vec would outperform the Jaccard coefficient because the Node2Vec method involves a shallow embedding where the model learns detailed embeddings for each node in the dataset. Since the dataset is only 5763 nodes, we are able to use shallow embeddings because we can store all of the embeddings. For the non-embedding method, I picked the Jaccard coefficient because it does better than common neighbors at accounting for the number of total neighbors a node has. I also chose the Jaccard coefficient because I wanted a local measure for link prediction. In contrast with the node embeddings from Node2Vec which are generated through random walks, the Jaccard coefficient only looks at direct neighbors for a node.
</p>
<p>The Node2Vec method was a more complex approach that involved training a model to optimize for nodes with similar embeddings appearing often on the same random walks. Prior to training, I dropped the properties column included in the nodes.csv file. In future iterations of this project, I would include the properties column within the embeddings for each of the nodes so that performance in link prediction would improve. Additionally, for future improvements to the model, I would treat the graph as a heterogeneous graph since its structure lends well to this design. By treating the graph as homogenous, it simplified the training process and interpretation of results, but performance of the link prediction suffered as a result.  
</p>

### Performance
<p>Surprisingly, the method that used the Jaccard coefficient outperformed the Node2Vec embedding method. I used several metrics from sklearn.metrics to compare the performance of the two approaches. 
</p>

**Node2Vec Embedding Method Performance Metrics**

| Validation Set Metrics | Value | 
| ------ | ------|
| Accuracy | 0.70 |
| Recall | 0.59 |
| F1-Score | 0.66 |
| ROC AUC Score | 0.68 |
| Average Precision | 0.77 |

<br>

| Test Set Metrics | Value | 
| ------ | ------|
| Accuracy | 0.69 |
| Recall | 0.60 |
| F1-Score | 0.66 |
| ROC AUC Score | 0.68 |
| Average Precision | 0.77 |




**Jaccard Coefficient Method Performance Metrics**

| Validation Set Metrics | Value | 
| ------ | ------|
| Accuracy | 0.83 |
| Recall | 0.71 |
| F1-Score | 0.80 |
| ROC AUC Score | 0.81 |
| Average Precision | 0.71 |

<br>

| Test Set Metrics | Value | 
| ------ | ------|
| Accuracy | 0.81 |
| Recall | 0.70 |
| F1-Score | 0.79 |
| ROC AUC Score | 0.79 |
| Average Precision | 0.68 |

<p>I did not expect the accuracy of the Jaccard coefficient method to be better than the Node2Vec random walk embedding method. I think there were two main reasons for the underperformance of the Node2Vec method relative to the Jaccard Coefficient. First, I did not use any node features inherently tied to each of the nodes with the embedding method. By dropping the properties column, the Node2Vec method learned graph structure, but it didn’t learn the semantic differences between the heterogenous nodes and edges. Without utilizing the information included in each of the nodes, the embeddings lacked the semantic understanding to learn when a dataset and science keyword should have been connected.
</p>
<p>
Second, I think this experiment reflects the predictive power of local neighborhood overlap. With the Jaccard coefficient that only takes into account the node degrees and the common neighbors, we are able to achieve relatively high accuracy in predicting the presence of a link between two nodes of different types. I assume that other local neighborhood overlap measures like common neighbors and Adamic-Adar index would perform similarly to this method, but I am curious how a global neighborhood measure like the Katz Index would perform. 
</p>

### Future Optimizations and Dataset Improvements
<p>In future link prediction tasks, I would like to try different hyperparameter combinations for the Node2Vec random walks. I did not try different combinations of the p and q parameters, but I hypothesize that by decreasing p and increasing q, I’d be able to bias the random walks to focus more on the local neighborhood for each node. This could result in improved performance similar to the Jaccard Coefficient results. I’d also like to try reducing the walk length parameter (which could help to focus on more local graph structure) and increase the embedding dimension. With bigger embeddings, it is possible we’d see better performance from the embedding method.
</p>
<p>As previously mentioned, I think my biggest improvement to the modeling approach would be to treat the graph as a heterogeneous graph and to include the properties column in my node embeddings so the model could use this information when predicting the presence of a link between two nodes. With limited link prediction and modeling experience, I did not have time to include these two improvements in my model and chose to use the relatively simple Node2Vec method to generate node embeddings.
</p>
<p>
For the dataset, I think the validation set and test set should have more types of edge relationships that are included in the training set rather than only the Dataset HAS_SCIENCEKEYWORD edge. Additionally, I think there should be more edge types that link to the Publication node type. With more of these relationships, we’d be able to better link authors of publications with potential future datasets that could be of interest to them and other publications in a similar field. Lastly, a quality of life improvement would be to include the presence of negative edges for modeling, but these were easily generated. For a larger graph, this may have been a larger computational barrier, but with less than 6000 nodes, the negative sampling was a quick task.  
</p>


